# TEST FACE RECOGNITION

In [1]:
import os
import re
import cv2
import time
import pickle
import numpy as np
from sklearn import neighbors

## PARAS

In [143]:
LWF_FOLDER_FACE = '/Users/khoa1799/Documents/DATA/LWF/data'
LWF_CUT_FOLDER_FACE = '/Users/khoa1799/Documents/DATA/LWF/cut_data'

LWF_PATH_USER_ID = '/Users/khoa1799/Documents/E-Healthcare-System/test_face_recognition/model/LWF_ID_Face'
LWF_PATH_USER_IMG_ENCODED = '/Users/khoa1799/Documents/E-Healthcare-System/test_face_recognition/model/LWF_Encoded_Face'
LWF_KNN_MODEL_PATH = "/Users/khoa1799/Documents/E-Healthcare-System/test_face_recognition/model/LWF_knn_clf_model.clf"
LWF_SVM_MODEL_PATH = "/Users/khoa1799/Documents/E-Healthcare-System/test_face_recognition/model/LWF_svm_clf_model.clf"

PREDICTOR_5_POINT_MODEL = '/Users/khoa1799/Documents/E-Healthcare-System/test_face_recognition/model/shape_predictor_5_face_landmarks.dat'
RESNET_MODEL = '/Users/khoa1799/Documents/E-Healthcare-System/test_face_recognition/model/dlib_face_recognition_resnet_model_v1.dat'

IMAGE_SIZE = 150

## FACE ENCODING

In [144]:
import dlib
pose_predictor_5_point = dlib.shape_predictor(PREDICTOR_5_POINT_MODEL)
face_encoder = dlib.face_recognition_model_v1(RESNET_MODEL)

In [145]:
def face_encodings(face_image, known_face_locations):
    raw_landmarks = _raw_face_landmarks(face_image, known_face_locations)
    return [np.array(face_encoder.compute_face_descriptor(face_image, raw_landmark_set, 1)) for raw_landmark_set in raw_landmarks]

def _css_to_rect(css):
    return dlib.rectangle(css[3], css[0], css[1], css[2])

def _raw_face_landmarks(face_image, face_locations):
    if face_locations is None:
        face_locations = _raw_face_locations(face_image)
    else:
        face_locations = [_css_to_rect(face_location) for face_location in face_locations]

    pose_predictor = pose_predictor_5_point

    return [pose_predictor(face_image, face_location) for face_location in face_locations]

import cv2
img = cv2.imread(LWF_FOLDER_FACE+ '/Donald_Fehr'+'/Donald_Fehr_0004.jpg')
print(img.shape)
test = face_encodings(img, [(0,250,250,0)])[0]

(250, 250, 3)


## GET THE BOUNDING BOX CONTAINNING FACE IN IMAGE BUT NOT ALIGNED

In [146]:
def Cut_Img(loaded_img):
    ret_img = loaded_img.copy()
    
    max_fra = max( loaded_img.shape[0], loaded_img.shape[1] ) / 320
    new_height = int( loaded_img.shape[0] / max_fra )
    new_width = int( loaded_img.shape[1] / max_fra )

    resized_img = cv2.resize(loaded_img, ( new_width, new_height ))
    RGB_resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(RGB_resized_img)
    for (top, right, bottom, left) in face_locations:
        top *= max_fra
        bottom *= max_fra
        left *= max_fra
        right *= max_fra

        return len(face_locations), ret_img[int(top):int(bottom),int(left):int(right)]
    return 0, 0

## LOAD AND SAVE MODEL

In [155]:
def SaveData(known_face_IDs, known_face_encodings, path_user_id, path_user_img_encoded):
    with open(path_user_id, mode='wb') as fp_1:
        pickle.dump(known_face_IDs, fp_1)

    with open(path_user_img_encoded, 'wb') as fp_2:
        pickle.dump(known_face_encodings, fp_2)

def SaveKNNModel(knn_clf, path_model):
    if path_model is not None:
        with open(path_model, 'wb') as f:
            pickle.dump(knn_clf, f)

def SaveSVMModel(svm_clf, path_model):
    if path_model is not None:
        with open(path_model, 'wb') as f:
            pickle.dump(svm_clf, f)

def LoadData(path_user_id, path_user_img_encoded):
    known_face_IDs = None
    known_face_encodings = None
    if not os.path.exists(path_user_id):
        print("There is no user id face to load")
        exit(-1)

    with open (path_user_id, 'rb') as fp_1:
        known_face_IDs = pickle.load(fp_1)

    if not os.path.exists(path_user_img_encoded):
        print("There is no user encoding face to load")
        exit(-1)

    with open (path_user_img_encoded, 'rb') as fp_2:
        known_face_encodings = pickle.load(fp_2)
    
    return known_face_IDs, known_face_encodings
        
def LoadKNNModel(path_model):
    if not os.path.exists(path_model):
        print("There is KNN model to load")
        exit(-1)

    with open(path_model, 'rb') as f:
        knn_clf = pickle.load(f)
        return knn_clf

def LoadSVMModel(path_model):
    if not os.path.exists(SVM_MODEL_PATH):
        print("There is KNN model to load")
        exit(-1)

    with open(SVM_MODEL_PATH, 'rb') as f:
        svm_clf = pickle.load(f)
        return svm_clf

# LWF DATASET

## Adjust brightness

In [148]:
import cv2
import numpy as np
BASE_BRIGHTNESS = 180
IMAGE_SIZE = 150

def adjust_brightness(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    v = hsv_img[:, :, 2]
    mean_v = np.mean(v)
    diff = BASE_BRIGHTNESS - mean_v
                   
    if diff < 0:
        v = np.where(v < abs(diff), v, v + diff)
    else:
        v = np.where( v + diff > 255, v, v + diff)

    hsv_img[:, :, 2] = v
    ret_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
    # return BRG image
    return ret_img

# TEST LWF DATASET

In [2]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

In [3]:
lfw_people = fetch_lfw_people(min_faces_per_person=5, resize=1, color=True)

### Create test and train dataset

In [4]:
import cv2
import matplotlib.pyplot as plt

# List BGR image
X = lfw_people.images
X = [cv2.normalize(src=x, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)for x in X]
X = [cv2.cvtColor(x, cv2.COLOR_RGB2BGR) for x in X]

Y = lfw_people.target
Y = [y for y in Y]

# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=20)
print("Number of train images: {}".format(len(X_train)))
print("Number of test images: {}".format(len(X_test)))

Number of train images: 4488
Number of test images: 1497


In [160]:
print("Shape of each images: {}".format(X[0].shape))

Shape of each images: (125, 94, 3)


In [7]:
freq = {}
max_people = 0
return_face_id = None
for user_id in y_test: 
    if (user_id in freq): 
        freq[user_id] += 1
    else: 
        freq[user_id] = 1
        
print(len(freq))

374


### CREATE EMBEDDED CODE AND USER ID

In [161]:
known_face_IDs = []
known_face_encodings = []
IMAGE_SIZE = 150

time_start = time.time()
for i in range(len(X_train)):
    img = X_train[i]
    user_ID = y_train[i]
    
    resized_img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
    resized_img = adjust_brightness(resized_img)
    RGB_resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)

    pre_proc_img = RGB_resized_img
    embedded_face = face_encodings(pre_proc_img, [(0,IMAGE_SIZE,IMAGE_SIZE,0)])[0]

    known_face_encodings.append(embedded_face)
    known_face_IDs.append(user_ID)

SaveData(known_face_IDs, known_face_encodings, LWF_PATH_USER_ID, LWF_PATH_USER_IMG_ENCODED)
print("Time load data {}".format(time.time() - time_start))

Time load data 102.51524925231934


## TRAIN KNN FOR LWF

In [162]:
# KNN PARAs
NUM_NEIGHBROS = 5
KNN_ALGORITHM = 'ball_tree'
KNN_WEIGHTS = 'distance'

knn_clf = None
known_face_IDs = []
known_face_encodings = []

# Load data
known_face_IDs, known_face_encodings = LoadData(LWF_PATH_USER_ID, LWF_PATH_USER_IMG_ENCODED)

# Create and train the KNN classifier
time_request = time.time()
knn_clf = neighbors.KNeighborsClassifier(n_neighbors=NUM_NEIGHBROS, algorithm=KNN_ALGORITHM, weights=KNN_WEIGHTS, metric='euclidean')

# self.__known_face_encodings is list of ndarray
# self.__known_face_IDs is list of str
knn_clf.fit(known_face_encodings, known_face_IDs)
SaveKNNModel(knn_clf, LWF_KNN_MODEL_PATH)
print("Finishing train KNN Model: {} second".format(time.time() - time_request))

Finishing train KNN Model: 0.050067901611328125 second


## TEST FACE RECOGNITION FOR LWF DATASET

In [165]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import time
import numpy as np
import os

THRESHOLD_FACE_REC = 0.5
knn_clf = None
known_face_IDs = None
known_face_encodings = None

dis_known_face = []
predict_known_face = []
list_time_known_faces = []
total_faces = 0

# Load KNN Model
knn_clf = LoadKNNModel(LWF_KNN_MODEL_PATH)
known_face_IDs, known_face_encodings = LoadData(LWF_PATH_USER_ID, LWF_PATH_USER_IMG_ENCODED)

for i in range(len(X_test)):
    img = X_test[i]
    
    # Pre-processing
    resized_img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
    resized_img = adjust_brightness(resized_img)
    RGB_resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
    
    # Convert to embedded code
    pre_proc_img = RGB_resized_img
    embedded_face = face_recognition.face_encodings(pre_proc_img, [(0,IMAGE_SIZE,IMAGE_SIZE,0)])[0]
    embedded_face = embedded_face.reshape(1,-1)
    
    has_face = False
    total_faces += 1
    start_time = time.time()
    
    # Classifying
    closet_distances = knn_clf.kneighbors(embedded_face, n_neighbors = 3)
    face_id = knn_clf.predict(embedded_face)
    meet_condition_threshold = [closet_distances[0][0][i] <= THRESHOLD_FACE_REC for i in range(len(closet_distances[0][0]))]
    
#     print(closet_distances)
#     print(face_id[-1])
    
    for i in range(len(meet_condition_threshold)):
        if meet_condition_threshold[i] and known_face_IDs[closet_distances[1][0][i]] == face_id[-1]:
            predict_known_face.append(face_id[-1])
            dis_known_face.append(closet_distances[0][0][i])
            has_face = True
            break
    if has_face == False:
        predict_known_face.append(-1)
        
    list_time_known_faces.append(time.time() - start_time)
    
    if total_faces > 1000:
        break

# print(predict_known_face)
# print(y_test[0:total_faces])
accurancy_score = accuracy_score(y_test[0:total_faces], predict_known_face)
precision_score = precision_score(y_test[0:total_faces], predict_known_face, average='weighted')
recall_score = recall_score(y_test[0:total_faces], predict_known_face, average='weighted')

means = np.mean(list_time_known_faces)
std = np.std(list_time_known_faces)
total_time = np.sum(list_time_known_faces)

print("Mean distance: {}".format(np.mean(dis_known_face)))
print("Max distance: {}".format(np.max(dis_known_face)))
print("Min distance: {}".format(np.min(dis_known_face)))
print()
print("Statistic report for identifying known face:")

print("\tAccurancy score: {}".format(accurancy_score))
print("\tPrecision score: {}".format(precision_score))
print("\tRecall score: {}".format(recall_score))
print()
print("\tTotal images: {}".format(total_faces))
print("\tTotal time: {}".format(total_time))
print("\tMeans time: {}".format(means))
print("\tStandard deviation time: {}".format(std))

Mean distance: 0.35181051751986525
Max distance: 0.49969589316406
Min distance: 0.0919298435723819

Statistic report for identifying known face:
	Accurancy score: 0.949050949050949
	Precision score: 0.9562814875039402
	Recall score: 0.949050949050949
	Total images: 1001
	Total time: 3.003488302230835
	Means time: 0.0030004878144164185
	Standard deviation time: 0.00036678161859896616


/Users/khoa1799/.virtualenvs/my_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/khoa1799/.virtualenvs/my_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
